Import Necessary Libraries

In [391]:
import pandas as pd
import tempfile
import os
from datetime import date as d
from datetime import timedelta

Set Up API

In [392]:
import statsapi as api

Set Up Beautiful Soup

In [393]:
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer

Set Up Selenium

In [394]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

For killing background chrome processes:

In [395]:
#import psutil
#import subprocess
#import signal

#def kill_chrome_processes():
#    for proc in psutil.process_iter():
#        if 'chrome' in proc.name() or 'chromedriver' in proc.name:
#            proc.kill()

Input Player Name and Date


NEED TO CHECK AND FIX ALL CELLS: IF STATS FOR SOMETHING DO NOT EXIST NEED TO DEAL WITH THAT!

ALSO NEED TO ENSURE CORRECT VALUES ARE PASSED IN FOR COMPUTING CORRECT VALUES

NEED TO CHECK PLAYER AVG OVER X GAMES - NEED TO COMPUTE PLAYER AVG OVER LAST GAME FOR PAST SEASONS AS WELL

NEED TO ENSURE THAT ALL DATA IS OF CORRECT TYPE, CORRECT VALUES, CORRECT YEARS, AND IF NONE NEEDS TO BE HANDLED

FIX CURRENT SET UP FOR COMPUTING AVERAGES. SHIT IS BUGGY AS FUCK

IF IT APPEARS TO BE GOOD: THERE WILL BE :P NEXT TO IT. IF NEEDS ADJUSTMENTS, VERIFICATION, OR REWORK, :( NEXT TO IT

ESPECIALLY DECIDE HOW TO DEAL WITH EDGE CASES AND DIVISION BY 0.

WALKS, HIT BY PITCH, AND SAC FLIES DO NOT COUNT AS AN AT BAT

TOTAL BASES ARE ONLY BASES REACHED VIA HITS.

In [396]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
#date = input('Enter Date: ')
date = '2024-07-02'
year = int(date[:4])
last_year = year - 1
print(year)
player_name = f'{first_name.lower()} {last_name.lower()}'


2024


fetch player_id :P

In [397]:
player_info = api.lookup_player(player_name, season=year)
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
print(player_id)
print(team_id)

660271
119


Gather batter_hand, batter_age, , batter_strike_zone_top, batter_strike_zone_bottom :P

In [398]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
batter_age = person_info['people'][0]['currentAge']
batter_strike_zone_top = person_info['people'][0]['strikeZoneTop']
batter_strike_zone_bottom = person_info['people'][0]['strikeZoneBottom']
batter_starting_year = int(person_info['people'][0]['mlbDebutDate'][:4])
batter_year = year - batter_starting_year


print(batter_age)
print(batter_year)
print(batter_hand)
print(batter_strike_zone_top)
print(batter_strike_zone_bottom)

30
6
L
3.49
1.6


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand :P

In [399]:
game_info = api.schedule(date=date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

if sPitcher_name == '':
    sPitcher_name = None
    sPitcher_id = None
    sPitcher_hand = None
else:
    sPitcher_info = api.lookup_player(sPitcher_name)
    sPitcher_id = sPitcher_info[0]['id']
    sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
    sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']
venue_name = game_info[0]['venue_name']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

print(game_status)
print(game_pk)
print(opp_id)
print(sPitcher_name)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_name)
print(venue_id)
print(home)

True
746133
109
Joe Mantiply
573009
L
Dodger Stadium
22
True


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified :(

What should happen when this data isnt uploaded yet?

In [400]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

temp = None
wind_speed = None
game_time = None

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])
            

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])
            

    if x.get('label') == 'First pitch':
        time_str = x['value'].strip('.')
        time_part, period = time_str.split(' ')
        hour_str, min_str = time_part.split(':')

        if period.lower() == 'am':
            game_time_str = f'{hour_str}{min_str}'
        
        elif period.lower() == 'pm':
            game_time_str = f'{int(hour_str) + 12}{min_str}'
        
        game_time = int(game_time_str)


    if x.get('label') == 'Umpires':
        ump_info = x['value']
        proj_ump_ids = []
        proj_umps = []
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        proj_umps.append(hp_ump)
                        proj_ump_ids.append(hp_ump_id)

        else:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_umps.append(ump)
                        proj_ump_ids.append(ump_id)

print(temp)
print(wind_speed)
print(game_time)
print(proj_ump_ids)
print(proj_umps)



75
8
1910
[605670]
['Dan Merzel']


fetch player game data: :P
- batting position

In [401]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

batting_position = None

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])

print (batting_position)

1


fetch game player stats :P
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [402]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    tb = game_stats.get('totalBases')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    tb = None
    k = None
    bb = None


print(f'ab: {ab}')
print(f'h: {h}')
print(f'r: {r}')
print(f'rbi: {rbi}')
print(f'hr: {hr}')
print(f'tb: {tb}')
print(f'k: {k}')
print(f'bb: {bb}')


ab: 5
h: 3
r: 2
rbi: 2
hr: 1
tb: 7
k: 2
bb: 0


Determine Whether Most Recent Streak Averages, or Historical Streak Averages are Needed for Player

Format Player URL for baseballreference.com

In [403]:
initial = last_name[:1].lower()

full_name = f'{first_name} {last_name}'.lower()

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                formatted_player_key = player_link[11:20]

print(formatted_player_key)

ohtansh01


Collect Data for Games Played Up to Desired Date
    - For entirely current year, or entirely last year

Format Player Game Data URL

In [404]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

if date == season_start_date:
    date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'
else:
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={year}#batting_gamelogs'

Set Correct Date

In [405]:
day = int(date[8:10])
month = int(date[5:7])

date_dt = d(year, month, day)

new_date_dt = date_dt - timedelta(days=1)

date = new_date_dt.strftime("%Y-%m-%d")

day = int(date[8:10])
month = int(date[5:7])

print(date)
print(day)
print(month)

2024-07-01
1
7


Scrape Player Game Data

In [406]:
response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    df = pd.DataFrame(data)

    df.rename(columns={
        'batting_avg': 'BA',
        'onbase_perc': 'OBP',
        'slugging_perc': 'SLG',
    }, inplace=True)

    all_columns = df.columns.tolist()

    columns_to_keep = ['game_num', 'date_game', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

    columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

    correct_columns_df = df.drop(columns_to_drop, axis=1)

    columns_to_convert_int = ['game_num', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

    columns_to_convert_float = ['BA', 'OBP', 'SLG']

    for column in columns_to_convert_int:
         correct_columns_df[column] = correct_columns_df[column].astype(int)

    for column in columns_to_convert_float:
        correct_columns_df[column] = correct_columns_df[column].astype(float)

    correct_columns_df['TB'] = correct_columns_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

    column_order = ['game_num', 'date_game', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
    correct_columns_df = correct_columns_df[column_order]

print(correct_columns_df)


    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
0          1  2024-03-20   5  2  0    1   0   0   0   2   0   0  0.400  0.400   
1          2  2024-03-21   5  1  1    1   0   0   0   1   0   0  0.300  0.273   
2          3  2024-03-28   3  2  1    0   1   0   0   3   1   1  0.385  0.400   
3          4  2024-03-29   4  0  0    0   0   0   0   0   0   1  0.294  0.316   
4          5  2024-03-30   5  1  1    0   0   0   0   1   1   2  0.273  0.320   
5          6  2024-03-31   4  1  1    0   1   0   0   2   0   2  0.269  0.310   
6          7  2024-04-01   4  1  1    1   1   0   0   2   0   1  0.267  0.303   
7          8  2024-04-02   3  0  0    0   0   0   0   0   1   1  0.242  0.297   
8          9  2024-04-03   4  2  2    1   0   0   1   5   0   1  0.270  0.317   
9         10  2024-04-05   5  2  2    2   1   0   1   6   0   0  0.286  0.326   
10        11  2024-04-06   4  2  1    0   0   0   0   2   1   1  0.304  0.353   
11        12  2024-04-07   4

Season Averages Up to Desired Date

In [407]:
season_stats = {}

game_data = correct_columns_df.iloc[-1]

games_played = game_data['game_num']

print(games_played)

season_stats['ab_per_game'] = correct_columns_df['AB'].mean()
season_stats['h_per_game'] = correct_columns_df['H'].mean()
season_stats['r_per_game'] = correct_columns_df['R'].mean()
season_stats['rbi_per_game'] = correct_columns_df['RBI'].mean()
season_stats['hr_per_game'] = correct_columns_df['HR'].mean()
season_stats['tb_per_game'] = correct_columns_df['TB'].mean()
season_stats['bb_per_game'] = correct_columns_df['BB'].mean()
season_stats['k_per_game'] = correct_columns_df['SO'].mean()
season_stats['ba_szn'] = game_data['BA']
season_stats['obp_szn'] = game_data['OBP']
season_stats['slg_szn'] = game_data['SLG']

season_stats_df = pd.DataFrame([season_stats])

print(season_stats_df)

82
   ab_per_game  h_per_game  r_per_game  rbi_per_game  hr_per_game  \
0     3.939024    1.243902    0.817073      0.756098     0.317073   

   tb_per_game  bb_per_game  k_per_game  ba_szn  obp_szn  slg_szn  
0          2.5      0.54878    0.926829   0.316    0.399    0.635  


Streak Average Data Collection

In [408]:
if games_played < 30:
    last_season_end_date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']

    day = int(last_season_end_date[8:10])
    month = int(last_season_end_date[5:7])

    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'

    response = requests.get(search_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

    if table_rows:

        data = []

        for row in table_rows:

            row_data = {}

            th = row.find('th')
            
            row_num = th.text
            
            row_data['game_num'] = row_num

            for td in row.find_all('td'):

                stat_type = td['data-stat']

                if stat_type == 'date_game':
                    stat_value = td['csk'][:10]
                else:
                    stat_value = td.text

                row_data[stat_type] = stat_value

            if row_data:

                game_day = int(row_data['date_game'][8:10])
                game_month = int(row_data['date_game'][5:7])

                if game_month < month:
                        data.append(row_data)
                elif game_month == month:
                    if game_day <= day:
                        data.append(row_data)

        df = pd.DataFrame(data)

        df.rename(columns={
            'batting_avg': 'BA',
            'onbase_perc': 'OBP',
            'slugging_perc': 'SLG',
        }, inplace=True)

        all_columns = df.columns.tolist()

        columns_to_keep = ['game_num', 'date_game', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

        columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

        last_season_df = df.drop(columns_to_drop, axis=1)

        columns_to_convert_int = ['game_num', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

        columns_to_convert_float = ['BA', 'OBP', 'SLG']

        for column in columns_to_convert_int:
            last_season_df[column] = last_season_df[column].astype(int)

        for column in columns_to_convert_float:
            last_season_df[column] = last_season_df[column].astype(float)

        last_season_df['TB'] = last_season_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        column_order = ['game_num', 'date_game', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
        last_season_df = last_season_df[column_order]

    print(last_season_df)




Last 7 Averages:

In [409]:
if games_played < 7:
    carry_over_games = 7 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_7_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_7_game_num = games_played - 7
    last_7_df = correct_columns_df.iloc[last_7_game_num:]

print(last_7_df)

last_7_stats = {}

last_7_stats['ab_last_7'] = last_7_df['AB'].mean()
last_7_stats['h_last_7'] = last_7_df['H'].mean()
last_7_stats['r_last_7'] = last_7_df['R'].mean()
last_7_stats['rbi_last_7'] = last_7_df['RBI'].mean()
last_7_stats['hr_last_7'] = last_7_df['HR'].mean()
last_7_stats['tb_last_7'] = last_7_df['TB'].mean()
last_7_stats['bb_last_7'] = last_7_df['BB'].mean()
last_7_stats['k_last_7'] = last_7_df['SO'].mean()
last_7_stats['ba_last_7'] = round(last_7_df['H'].sum() / last_7_df['AB'].sum(), 3)
last_7_stats['obp_last_7'] = round((last_7_df['H'].sum() + last_7_df['BB'].sum() + last_7_df['HBP'].sum()) / last_7_df['AB'].sum(), 3)
last_7_stats['slg_last_7'] = round( last_7_df['TB'].sum() / last_7_df['AB'].sum(), 3)

last_7_avg_df = pd.DataFrame([last_7_stats])

print(last_7_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
75        76  2024-06-22   4  1  1    2   0   0   1   4   1   0  0.321  0.398   
76        77  2024-06-24   3  0  0    1   0   0   0   0   1   2  0.318  0.395   
77        78  2024-06-25   4  2  2    2   0   0   1   5   1   2  0.320  0.398   
78        79  2024-06-26   2  1  2    1   0   0   1   4   2   0  0.322  0.402   
79        80  2024-06-28   3  1  0    0   0   0   0   1   1   1  0.322  0.403   
80        81  2024-06-29   4  1  2    1   0   0   1   4   2   2  0.321  0.405   
81        82  2024-06-30   5  0  0    0   0   0   0   0   0   3  0.316  0.399   

      SLG  HBP  
75  0.632    0  
76  0.626    0  
77  0.634    0  
78  0.643    0  
79  0.640    0  
80  0.645    0  
81  0.635    0  
   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0   3.571429  0.857143       1.0         1.0   0.571429   2.571429   1.142857   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  
0  

Last 15 Averages

In [410]:
if games_played < 15:
    carry_over_games = 15 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_15_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_15_game_num = games_played - 15
    last_15_df = correct_columns_df.iloc[last_15_game_num:]

print(last_15_df)

last_15_stats = {}

last_15_stats['ab_last_15'] = last_15_df['AB'].mean()
last_15_stats['h_last_15'] = last_15_df['H'].mean()
last_15_stats['r_last_15'] = last_15_df['R'].mean()
last_15_stats['rbi_last_15'] = last_15_df['RBI'].mean()
last_15_stats['hr_last_15'] = last_15_df['HR'].mean()
last_15_stats['tb_last_15'] = last_15_df['TB'].mean()
last_15_stats['bb_last_15'] = last_15_df['BB'].mean()
last_15_stats['k_last_15'] = last_15_df['SO'].mean()
last_15_stats['ba_last_15'] = round(last_15_df['H'].sum() / last_15_df['AB'].sum(), 3)
last_15_stats['obp_last_15'] = round((last_15_df['H'].sum() + last_15_df['BB'].sum() + last_15_df['HBP'].sum()) / last_15_df['AB'].sum(), 3)
last_15_stats['slg_last_15'] = round( last_15_df['TB'].sum() / last_15_df['AB'].sum(), 3)

last_15_avg_df = pd.DataFrame([last_15_stats])

print(last_15_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
67        68  2024-06-14   4  2  0    0   0   0   0   2   0   0  0.309  0.379   
68        69  2024-06-15   3  0  0    0   0   0   0   0   1   0  0.305  0.377   
69        70  2024-06-16   3  2  2    2   0   0   2   8   1   0  0.309  0.382   
70        71  2024-06-17   5  3  2    1   2   0   0   5   1   0  0.314  0.388   
71        72  2024-06-18   5  2  2    2   0   0   1   5   0   0  0.316  0.388   
72        73  2024-06-19   5  2  1    3   1   0   0   3   0   1  0.317  0.388   
73        74  2024-06-20   3  1  1    1   0   0   1   4   2   1  0.318  0.391   
74        75  2024-06-21   2  2  1    2   0   0   1   5   2   0  0.322  0.398   
75        76  2024-06-22   4  1  1    2   0   0   1   4   1   0  0.321  0.398   
76        77  2024-06-24   3  0  0    1   0   0   0   0   1   2  0.318  0.395   
77        78  2024-06-25   4  2  2    2   0   0   1   5   1   2  0.320  0.398   
78        79  2024-06-26   2

Last 30 Averages

In [411]:
if games_played < 30:
    carry_over_games = 30 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_30_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_30_game_num = games_played - 30
    last_30_df = correct_columns_df.iloc[last_30_game_num:]

print(last_30_df)

last_30_stats = {}

last_30_stats['ab_last_30'] = last_30_df['AB'].mean()
last_30_stats['h_last_30'] = last_30_df['H'].mean()
last_30_stats['r_last_30'] = last_30_df['R'].mean()
last_30_stats['rbi_last_30'] = last_30_df['RBI'].mean()
last_30_stats['hr_last_30'] = last_30_df['HR'].mean()
last_30_stats['tb_last_30'] = last_30_df['TB'].mean()
last_30_stats['bb_last_30'] = last_30_df['BB'].mean()
last_30_stats['k_last_30'] = last_30_df['SO'].mean()
last_30_stats['ba_last_30'] = round(last_30_df['H'].sum() / last_30_df['AB'].sum(), 3)
last_30_stats['obp_last_30'] = round((last_30_df['H'].sum() + last_30_df['BB'].sum() + last_30_df['HBP'].sum()) / last_30_df['AB'].sum(), 3)
last_30_stats['slg_last_30'] = round( last_30_df['TB'].sum() / last_30_df['AB'].sum(), 3)

last_30_avg_df = pd.DataFrame([last_30_stats])

print(last_30_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
52        53  2024-05-26   4  1  1    0   0   0   0   1   0   1  0.336  0.403   
53        54  2024-05-28   5  0  0    0   0   0   0   0   0   2  0.329  0.395   
54        55  2024-05-29   5  2  1    3   0   0   1   5   0   1  0.330  0.395   
55        56  2024-05-31   3  0  0    0   0   0   0   0   1   1  0.326  0.393   
56        57  2024-06-01   3  1  0    0   0   0   0   1   1   1  0.326  0.395   
57        58  2024-06-02   3  0  0    0   0   0   0   0   1   2  0.322  0.392   
58        59  2024-06-04   4  1  0    0   0   0   0   1   0   2  0.321  0.390   
59        60  2024-06-05   5  2  1    2   0   0   1   5   0   2  0.322  0.390   
60        61  2024-06-06   6  1  1    0   0   0   0   1   0   3  0.318  0.385   
61        62  2024-06-07   5  0  1    0   0   0   0   0   0   0  0.312  0.379   
62        63  2024-06-08   4  1  2    1   0   0   0   1   1   1  0.311  0.379   
63        64  2024-06-09   4

Merge Streak Data

In [412]:
streak_data_df = pd.concat([last_7_avg_df, last_15_avg_df, last_30_avg_df], axis=1)
print(streak_data_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0   3.571429  0.857143       1.0         1.0   0.571429   2.571429   1.142857   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  ab_last_15  h_last_15  \
0  1.428571       0.24        0.56        0.72    3.666667   1.333333   

   r_last_15  rbi_last_15  hr_last_15  tb_last_15  bb_last_15  k_last_15  \
0   1.066667          1.2         0.6    3.333333         1.0        0.8   

   ba_last_15  obp_last_15  slg_last_15  ab_last_30  h_last_30  r_last_30  \
0       0.364        0.636        0.909    3.866667   1.066667   0.933333   

   rbi_last_30  hr_last_30  tb_last_30  bb_last_30  k_last_30  ba_last_30  \
0          0.9    0.433333         2.5         0.7   1.033333       0.276   

   obp_last_30  slg_last_30  
0        0.466        0.647  


Merge Season Averages and Streak Data



In [413]:
batter_season_stats_df = pd.concat([streak_data_df, season_stats_df], axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(batter_season_stats_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0   3.571429  0.857143       1.0         1.0   0.571429   2.571429   1.142857   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  ab_last_15  h_last_15  \
0  1.428571       0.24        0.56        0.72    3.666667   1.333333   

   r_last_15  rbi_last_15  hr_last_15  tb_last_15  bb_last_15  k_last_15  \
0   1.066667          1.2         0.6    3.333333         1.0        0.8   

   ba_last_15  obp_last_15  slg_last_15  ab_last_30  h_last_30  r_last_30  \
0       0.364        0.636        0.909    3.866667   1.066667   0.933333   

   rbi_last_30  hr_last_30  tb_last_30  bb_last_30  k_last_30  ba_last_30  \
0          0.9    0.433333         2.5         0.7   1.033333       0.276   

   obp_last_30  slg_last_30  ab_per_game  h_per_game  r_per_game  \
0        0.466        0.647     3.939024    1.243902    0.817073   

   rbi_per_game  hr_per_game  tb_per_game  bb_per_game  k_per_game  ba_szn  \
0      0.756

Fetch Starting Pitcher Season Data


MASSIVE:

PLAN FOR FUTURE FOR PITCHER, BVP, BVOPP, OPP, BVB Data

DESIGN THE 'Starting Pitcher Efficiency Index'

Step 1: SP Data
    - Gather Data such as:
        - ip per game
        - batters faced per game
        - plate appearances per game
        - pitches per game
        - at bats per game

Step 2: Include these new stat categories:
    - Batter Vs Opponent - Previous Batter stats from each game against the opponent this season
    - Team Vs Opponent Stats - Avg Team Stats for Each Final Boxscore of Previous Season Matchups
    - Pitcher vs Team Stats - Avg Pitcher Stats Vs Batter's team from previous mathcups
    - Batter Vs Pitcher - Batter Vs Pitcher Stats from IP or PA from each game featuring the two
    - Batter Vs Bullpen (Batter Vs Team - Batter Vs Pitcher)

    Goal is to determine how valuable the batter's plate appearences with the starting pitcher compared to the batter's plate appearances with the relievers and closers.
    What percentage of hits come from the starting pitcher vs the relievers. Want to try to predict the number of plate appearnaces the batter will get during a game with this starting pitcher, and then use this value to predict hits by comparing the likelihood the batter will get a hit within the plate appearances against the starting pitcher and the likelihood the batter will get a hit within the plate appearances against the relievers and closers. Then using these averages, predict how likely the batter is to get a hit in this game. Basically instead of just using starting pitcher, want to treat both the starting pitcher and other pitchers as an entire game to predict hits. This will be done by leveraging the innings pitched by the starter and other pitchers with the stats of starter v batter and bullpen v batter


Format Pitcher URL

In [414]:
sp_name_parts = sPitcher_name.split()

sp_first_name = sp_name_parts[0].lower()
sp_last_name = sp_name_parts[1].lower()

initial = sp_last_name[:1]

full_name = f'{sp_first_name} {sp_last_name}'

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                formatted_pitcher_key = player_link[11:20]

print(formatted_pitcher_key)

mantijo01


In [415]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_pitcher_key}&t=p&year={year}#pitching_gamelogs'

response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('pitching_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    sp_df = pd.DataFrame(data)

    sp_df.rename(columns={
            'earned_run_avg': 'ERA',
            'batters_faced': 'BF',
            'pitches': 'P',
            'strikes_total': 'S',
        }, inplace=True)

    sp_all_columns = sp_df.columns.tolist()

    sp_columns_to_keep = ['game_num', 'date_game', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'BF', 'P', 'S', 'AB', '2B', '3B']

    sp_columns_to_drop = [column for column in sp_all_columns if column not in sp_columns_to_keep]

    sp_column_df = sp_df.drop(sp_columns_to_drop, axis=1)

    sp_columns_to_convert_int = ['game_num', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'BF', 'P', 'S', 'AB', '2B', '3B']

    sp_columns_to_convert_float = ['IP', 'ERA']

    for column in sp_columns_to_convert_int:
        sp_column_df[column] = sp_column_df[column].astype(int)

    for column in sp_columns_to_convert_float:
        sp_column_df[column] = sp_column_df[column].astype(float)
    
    #print(sp_column_df)

    sp_column_df['WHIP'] = sp_column_df.apply(lambda row: ((row['H'] + row['BB']) / row['IP']) if row['IP'] > 0.0 else (row['H'] + row['BB']), axis=1)

    sp_column_df['TB'] = sp_column_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

    sp_column_df['IP'] = sp_column_df['IP'].apply(lambda ip: int(ip) + (1/3) if round(ip % 1, 1) == 0.1 else int(ip) + (2/3) if round(ip % 1, 1) == 0.2 else ip)

    sp_column_df['TO'] = sp_column_df.apply(lambda row: row['IP']*3, axis=1)

    sp_column_df['S%'] = sp_column_df.apply(lambda row: row['S'] / row['P'], axis=1)

    sp_column_df['BAA'] = sp_column_df.apply(lambda row: row['H'] / row['AB'], axis=1)

    sp_column_order = ['game_num', 'date_game', 'IP', 'P', 'S', 'S%', 'BF', 'AB', 'H', '2B', '3B', 'R', 'ER', 'HR', 'TB', 'BB', 'SO', 'TO', 'HBP', 'ERA', 'WHIP', 'BAA']
                       
    sp_complete_df = sp_column_df[sp_column_order]

    print(sp_complete_df)

    game_num   date_game        IP   P   S        S%  BF  AB  H  2B  3B  R  \
0          1  2024-03-29  1.000000  21  16  0.761905   6   6  3   0   1  2   
1          2  2024-04-01  1.666667  19  14  0.736842   6   6  1   1   0  0   
2          3  2024-04-03  1.000000  14   9  0.642857   3   3  0   0   0  0   
3          4  2024-04-05  1.000000  16  10  0.625000   4   3  0   0   0  0   
4          5  2024-04-08  0.333333   8   4  0.500000   3   2  1   0   0  0   
5          6  2024-04-10  0.333333   3   2  0.666667   1   1  0   0   0  0   
6          7  2024-04-12  0.666667  18  14  0.777778   4   4  2   0   1  1   
7          8  2024-04-14  1.333333  18   9  0.500000   5   4  0   0   0  0   
8          9  2024-04-15  0.333333   5   2  0.400000   1   1  0   0   0  0   
9         10  2024-04-17  0.333333  14   8  0.571429   5   4  3   0   0  2   
10        11  2024-04-20  0.666667   5   5  1.000000   3   3  1   0   0  0   
11        12  2024-04-22  0.333333   3   3  1.000000   1   1  0 

Pitcher Season Stats Up to Desired Date

In [416]:
sp_season_stats = {}

sp_game_data = sp_complete_df.iloc[-1]

sp_games_started = sp_game_data['game_num']

print(sp_games_started)

sp_season_stats['sp_games_started'] = sp_games_started
sp_season_stats['sp_ip_per_game'] = sp_complete_df['IP'].mean()
sp_season_stats['sp_p_per_game'] = sp_complete_df['P'].mean()
sp_season_stats['sp_s_per_game'] = sp_complete_df['S'].mean()
sp_season_stats['sp_s%'] = sp_complete_df['S'].sum() / sp_complete_df['P'].sum()
sp_season_stats['sp_bf_per_game'] = sp_complete_df['BF'].mean()
sp_season_stats['sp_ab_per_game'] = sp_complete_df['AB'].mean()
sp_season_stats['sp_h_per_game'] = sp_complete_df['H'].mean()
sp_season_stats['sp_r_per_game'] = sp_complete_df['R'].mean()
sp_season_stats['sp_er_per_game'] = sp_complete_df['ER'].mean()
sp_season_stats['sp_hr_per_game'] = sp_complete_df['HR'].mean()
sp_season_stats['sp_tb_per_game'] = sp_complete_df['TB'].mean()
sp_season_stats['sp_bb_per_game'] = sp_complete_df['BB'].mean()
sp_season_stats['sp_k_per_game'] = sp_complete_df['SO'].mean()
sp_season_stats['sp_k%'] = sp_complete_df['SO'].sum() / sp_complete_df['BF'].sum()
sp_season_stats['sp_to_per_game'] = sp_complete_df['TO'].mean()
sp_season_stats['sp_era'] = sp_game_data['ERA']
sp_season_stats['sp_whip_per_game'] = sp_complete_df['WHIP'].mean()
sp_season_stats['sp_whip_szn'] = ((sp_complete_df['H'].sum() + sp_complete_df['BB'].sum() ) / sp_complete_df['IP'].sum())
sp_season_stats['sp_baa'] = sp_complete_df['H'].sum() / sp_complete_df['AB'].sum()

sp_season_stats_df = pd.DataFrame([sp_season_stats])

print(sp_season_stats_df)

37
   sp_games_started  sp_ip_per_game  sp_p_per_game  sp_s_per_game    sp_s%  \
0                37        0.828829           12.0       8.162162  0.68018   

   sp_bf_per_game  sp_ab_per_game  sp_h_per_game  sp_r_per_game  \
0        3.351351        3.108108       0.702703       0.378378   

   sp_er_per_game  sp_hr_per_game  sp_tb_per_game  sp_bb_per_game  \
0        0.378378             0.0        1.054054        0.243243   

   sp_k_per_game     sp_k%  sp_to_per_game  sp_era  sp_whip_per_game  \
0       0.702703  0.209677        2.486486    4.11          3.684275   

   sp_whip_szn    sp_baa  
0     1.141304  0.226087  


Format Batter vs Starting Pitcher URL :P

In [417]:
formatted_batter_name = player_name.replace(' ', '%20')

team_schedule = api.schedule(start_date=season_start_date, end_date=date, team=team_id, opponent=opp_id)

if len(team_schedule) > 0:

    games_vs_opp = []
    games_vs_sp = []

    for game in team_schedule:
        if game['game_type'] != 'S' and game['status'] != 'Postponed':
            for log in api.get('person_stats', {'personId': player_id, 'gamePk': game['game_id']})['stats'][0]['splits']:
                if log.get('group') == 'hitting' and log['stat']:
                    games_vs_opp.append(game['game_id'])
                    if game['home_id'] == opp_id:
                        if game['home_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
                    elif game['away_id'] == opp_id:
                        if game['away_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
    
    num_games_vs_sp = len(games_vs_sp)
    num_games_vs_opp = len(games_vs_opp)

    game_strings = [f'%22{game_vs_opp}%22' for game_vs_opp in games_vs_opp]

    comma_game_strings =f'[' + ','.join(game_strings) + ']'

    if num_games_vs_sp == 0:
        bvp_url = None
    else:
        bvp_url = f'https://baseballsavant.mlb.com/illustrator?playerId={player_id}&playerType=batter&name={formatted_batter_name}&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22{sPitcher_id}%22]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=' + comma_game_strings + '&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7'


else:
    num_games_vs_sp = 0
    num_games_vs_opp = 0
    games_vs_opp = None
    games_vs_sp = None
    bvp_url = None

print(num_games_vs_sp)
print(num_games_vs_opp)

print(bvp_url)
print(games_vs_opp)
print(games_vs_sp)


1
5
https://baseballsavant.mlb.com/illustrator?playerId=660271&playerType=batter&name=shohei%20ohtani&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22573009%22]&balls=[]&battedBallType=[]&years=[2024]&strikes=[]&outs=[]&selectedGames=[%22747211%22,%22747210%22,%22746146%22,%22746145%22,%22746143%22]&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7
[747211, 747210, 746146, 746145, 746143]
['2024-05-20']


Gather Raw BVP Data

In [418]:
if bvp_url is not None:
    bvp_raw_data = {}

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")

    #service = Service(chrome_driver_path)
    #driver = webdriver.Chrome(service=service, options=options)
    
    with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
        try:
            driver.get(bvp_url)

            driver.implicitly_wait(10)

            stats_boxes = driver.find_elements(By.CLASS_NAME, 'stats-box')

            for stats_box in stats_boxes:
                stat_type = stats_box.find_element(By.CLASS_NAME, 'stats-title')
                stat_value = stats_box.find_element(By.CLASS_NAME, 'stats-value')
                bvp_raw_data[stat_type.text] = stat_value.text

        finally:
            driver.close()
            driver.quit()
            #kill_chrome_processes()

    for key in bvp_raw_data.keys():
        bvp_raw_data[key] = float(bvp_raw_data[key])

else:
    bvp_raw_data = None

print(bvp_raw_data)

{'Pitches': 9.0, 'PA': 4.0, 'H': 0.0, '1B': 0.0, '2B': 0.0, '3B': 0.0, 'HR': 0.0, 'AVG': 0.0, 'SO': 1.0, 'wOBA': 0.0, 'K%': 25.0, 'EV': 82.0, 'LA': -9.0}


Manipulate bvp data

TO DO: Remove normalization to 9 innings and keep as total per game.
Consider: switching to per plate appearance
Consider: instead of just looking at bvp, look at entire game where that pitcher was sp
            - record stats per plate appearance against starting pitcher
            - record stats per plate appearance against entire bullpen
            - most importantly, determine avg of plate appearances against sp per game and stats associated with those plate appearances
            - big idea: want to determine how many plate appearances the batter will get vs this sp, and then determine the likelihood of player getting a hit within those plate appearances.
            - Then want to determine how many plate appearances the batter will get after sp, then then determine the likelihood of a player getting a hit within those plate appearances.
            - Combine both of these stats to predict the amount of plate appearances the batter will get per game, and how likely they are to get a hit given the split of those plate appearances 

In [419]:
bvp_data = {'bvp_games_faced': 0,
            'bvp_sp_ip_per_game': 0.0,
            'bvp_sp_p_per_game': 0.0,
            'bvp_b_pa_per_game': 0.0,
            'bvp_b_h_per_game': 0.0,
            'bvp_b_hr_per_game': 0.0,
            'bvp_b_tb_per_game': 0.0,
            'bvp_b_k_per_game': 0.0,
            'bvp_sp_k%': 0.0,
            'bvp_sp_woba': 0.0,
            'bvp_b_ba': 0.0,
            }

if bvp_raw_data is not None:
    bvp_total_data = {}
    sp_ip = 0.0

    for d in games_vs_sp:
        sp_ip += float(sp_column_df[sp_column_df['date_game'] == d]['IP'].iloc[0])

    sp_ip_per_bvp = round(sp_ip / num_games_vs_sp, 3)
    bvp_total_data['bvp_sp_ip_per_game'] = sp_ip_per_bvp

    bvp_total_data['bvp_sp_p_per_game'] = round(bvp_raw_data['Pitches'] / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_pa_per_game'] = round(bvp_raw_data['PA'] / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_h_per_game'] = round(bvp_raw_data['H'] / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_1B_adj'] = round(bvp_raw_data['1B'] / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_2B_adj'] = round(bvp_raw_data['2B'] / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_3B_adj'] = round(bvp_raw_data['3B'] / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_hr_per_game'] = round(bvp_raw_data['HR'] / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_k_per_game'] = round(bvp_raw_data['SO'] / sp_ip_per_bvp, 3)
    
    bvp_total_data['bvp_b_tb_per_game'] = round((bvp_total_data['bvp_1B_adj'] + 2 * bvp_total_data['bvp_2B_adj']
                                                  + 3 * bvp_total_data['bvp_3B_adj'] + 4 * bvp_total_data['bvp_b_hr_per_game']), 3)

    bvp_data['bvp_b_ba'] = bvp_raw_data['AVG']
    bvp_data['bvp_sp_k%'] = bvp_raw_data['K%']
    bvp_data['bvp_sp_woba'] = bvp_raw_data['wOBA']

    for i in ['bvp_1B_adj', 'bvp_2B_adj', 'bvp_3B_adj']:
        bvp_total_data.pop(i)
    
    for total in bvp_total_data.keys():
        bvp_data[total] = round(bvp_total_data[total] / num_games_vs_sp, 3)
    
    bvp_data['bvp_games_faced'] = num_games_vs_sp
    
bvp_data_df = pd.DataFrame([bvp_data]).astype(float)

print(bvp_data_df)
        

   bvp_games_faced  bvp_sp_ip_per_game  bvp_sp_p_per_game  bvp_b_pa_per_game  \
0              1.0                 1.0                9.0                4.0   

   bvp_b_h_per_game  bvp_b_hr_per_game  bvp_b_tb_per_game  bvp_b_k_per_game  \
0               0.0                0.0                0.0               1.0   

   bvp_sp_k%  bvp_sp_woba  bvp_b_ba  
0       25.0          0.0       0.0  


Gather b_vs_opp Data

In [420]:
bv_opp_data_dict = {}

if num_games_vs_opp > 0:

    bv_opp_raw_data = []

    for game in games_vs_opp:
        game_vs_opp_log = api.get('person_stats', {'personId': player_id, 'gamePk': game})['stats'][0]['splits']
        game_vs_opp_data = {}
        for group in game_vs_opp_log:
            if group['group'] == 'hitting':
                for s in group['stat']:
                    game_vs_opp_data[s] = group['stat'][s]
        bv_opp_raw_data.append(game_vs_opp_data)
    
    bv_opp_raw_data_df = pd.DataFrame(bv_opp_raw_data)

    bv_opp_raw_data_df.rename(columns={
        'runs': 'bv_opp_r',
        'doubles': '2B',
        'triples': '3B',
        'homeRuns': 'bv_opp_hr',
        'strikeOuts': 'bv_opp_k',
        'baseOnBalls': 'bv_opp_bb',
        'hits': 'bv_opp_h',
        'hitByPitch': 'HBP',
        'atBats': 'bv_opp_ab',
        'totalBases': 'bv_opp_tb',
        'rbi': 'bv_opp_rbi',
        'sacFlies': 'SF'
    }, inplace=True)

    bv_opp_columns = bv_opp_raw_data_df.columns.to_list()

    bv_opp_columns_to_keep = ['bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', '2B', '3B', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'HBP', 'SF']

    bv_opp_columns_to_drop = [column for column in bv_opp_columns if column not in bv_opp_columns_to_keep]

    bv_opp_columns_df = bv_opp_raw_data_df.drop(bv_opp_columns_to_drop, axis=1)

    bv_opp_columns_df.astype(int)

    for column in bv_opp_columns_df.columns.to_list():
        bv_opp_data_dict[column] = bv_opp_columns_df[column].mean()

    bv_opp_data_dict['bv_opp_ba'] = round(bv_opp_data_dict['bv_opp_h'] / bv_opp_data_dict['bv_opp_ab'], 3)
    bv_opp_data_dict['bv_opp_obp'] = round((bv_opp_data_dict['bv_opp_h'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP']) / (
        bv_opp_data_dict['bv_opp_ab'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP'] + bv_opp_data_dict['SF']), 3)
    bv_opp_data_dict['bv_opp_slg'] = round(bv_opp_data_dict['bv_opp_tb'] / bv_opp_data_dict['bv_opp_ab'], 3)

    for key in ['2B', '3B', 'HBP', 'SF']:
        bv_opp_data_dict.pop(key)
    
    bv_opp_data_dict['bv_opp_games_played'] = num_games_vs_opp

else:
    bv_opp_data_dict = {'bv_opp_games_played': 0, 'bv_opp_ab': 0, 'bv_opp_h': 0,'bv_opp_r': 0, 'bv_opp_rbi': 0, 'bv_opp_hr': 0, 'bv_opp_tb': 0, 'bv_opp_bb': 0, 'bv_opp_k': 0,
                         'bv_opp_ba': 0.0, 'bv_opp_obp': 0.0, 'bv_opp_slg': 0.0}

bv_opp_data_df = pd.DataFrame([bv_opp_data_dict])

bv_opp_column_order = ['bv_opp_games_played', 'bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'bv_opp_ba', 'bv_opp_obp', 'bv_opp_slg']

bv_opp_df = bv_opp_data_df[bv_opp_column_order]

print(bv_opp_df)

   bv_opp_games_played  bv_opp_ab  bv_opp_h  bv_opp_r  bv_opp_rbi  bv_opp_hr  \
0                    5        4.0       1.2       0.8         0.4        0.0   

   bv_opp_tb  bv_opp_bb  bv_opp_k  bv_opp_ba  bv_opp_obp  bv_opp_slg  
0        1.4        0.4       0.8        0.3       0.364        0.35  


Fetch Umpire Stats
- If Home Plate Ump announced
    - Return data for only HP ump 
- If Home Plate Ump not yet announced
    - Fetch data for all 4 umps
    - hp_ump stats = avg of 4 umps stats

In [421]:
if len(proj_ump_ids) > 0:

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")

    #service = Service(chrome_driver_path)
    #driver = webdriver.Chrome(service=service, options=options)

    with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
        try:
            driver.get('https://swishanalytics.com/mlb/mlb-umpire-factors')

            table = driver.find_element(By.CSS_SELECTOR, "div.table-responsive.table-umpires > table")

            rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")

            umpire_data = []

            for row in rows:
                umpire_name = row.find_element(By.CSS_SELECTOR, "td.umps-name").text.strip()
                if umpire_name in proj_umps:
                    row_data = [cell.text.strip() for cell in row.find_elements(By.CSS_SELECTOR, "td")]
                    umpire_data.append(row_data[3:])
                    
        finally:
            driver.close()
            driver.quit()

    #print(umpire_data)

    for u in umpire_data:
        i = 0
        while i < len(u):
            if u[i].endswith('%'):
                u[i] = float(u[i].strip('%')) / 100
            elif u[i].endswith('x'):
                u[i] = float(u[i].strip('x'))
            else:
                u[i] = float(u[i])
            i += 1

    print(umpire_data)

else:
    umpire_data = [[None for _ in range(12)]]

    print(umpire_data)


[[0.235, 0.077, 8.46, 0.244, 0.311, 0.399, 1.04, 0.94, 0.99, 1.01, 1.0, 1.01]]


In [422]:
umpire_data_df = pd.DataFrame(columns=['ump_k%', 'ump_bb%', 'ump_r_per_game', 'ump_ba', 'ump_obp', 'ump_slg', 
                                       'ump_k_boost', 'ump_bb_boost', 'ump_r_boost', 'ump_ba_boost', 'ump_obp_boost', 'ump_slg_boost'])

if len(umpire_data) > 1:
    avg_list = [sum(x) / len(umpire_data) for x in zip(*umpire_data)]
    
    umpire_data_df.loc[0] = avg_list

else:
    umpire_data_df.loc[0] = umpire_data[0]

print(umpire_data_df)

    

   ump_k%  ump_bb%  ump_r_per_game  ump_ba  ump_obp  ump_slg  ump_k_boost  \
0   0.235    0.077            8.46   0.244    0.311    0.399         1.04   

   ump_bb_boost  ump_r_boost  ump_ba_boost  ump_obp_boost  ump_slg_boost  
0          0.94         0.99          1.01            1.0           1.01  


Fetch Venue Stats:

*FUTURE CHANGES: DETERMINE A BETTER WAY TO SET DATA FOR SWITCH HITTERS.

In [423]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--disable-images")
prefs = {
    'profile.default_content_setting_values': {
        'images': 2,
        'javascript': 2,
        'stylesheet': 2
    }
}
options.add_experimental_option('prefs', prefs)

#service = Service(chrome_driver_path)
#driver = webdriver.Chrome(service=service, options=options)

with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:

    try:
        driver.get('https://swishanalytics.com/mlb/mlb-park-factors')

        venue_element = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.XPATH, f"//h4[contains(text(), '{venue_name}')]")))

        venue_table = venue_element.find_element(By.XPATH, "./following::table[1]")

        venue_rows = venue_table.find_elements(By.TAG_NAME, 'tr')

        venue_data = []

        for row in venue_rows[1:]:
            venue_cells = row.find_elements(By.TAG_NAME, 'td')
            venue_row_data = [cell.text for cell in venue_cells]

            if batter_hand == 'L':
                venue_data.append(float(venue_row_data[0]))
            elif batter_hand == 'R':
                venue_data.append(float(venue_row_data[2]))
            else:
                venue_data.append(round((float(venue_row_data[0]) + float(venue_row_data[2])) / 2, 3))

    finally:
        driver.close()
        driver.quit()

print(venue_data)

[1.21, 0.5, 0.92, 0.99, 0.98, 0.95, 1.01, 0.98, 0.92]
